In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

In [2]:
df_train = pd.read_csv(r'train_car.csv')

In [3]:
df_train.shape

(4209, 378)

In [4]:
df_train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y_train = df_train['y'].values

In [6]:
cols = [c for c in df_train.columns if 'X' in c]
print('Number of features: {}'.format(len(cols)))
df_train[cols].dtypes.value_counts()

Number of features: 376


int64     368
object      8
dtype: int64

In [7]:
counts = [[], [], []]
for c in cols:
    typ = df_train[c].dtype
    uniq = len(np.unique(df_train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)

In [8]:
print('Constant features: {} Binary features: {} Categorical features: {}\n'
      .format(*[len(c) for c in counts]))
print('Constant features:', counts[0])
print('Categorical features:', counts[2])

Constant features: 12 Binary features: 356 Categorical features: 8

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [9]:
df_test = pd.read_csv(r'test_car.csv')

In [10]:
usable_columns = list(set(df_train.columns) - set(['ID', 'y']))
y_train = df_train['y'].values
id_test = df_test['ID'].values

In [11]:
x_train = df_train[usable_columns]
x_test = df_test[usable_columns]

In [12]:
# 2. Check for null and unique values for test and train sets.

if df_train.isnull().any().any()==False:
    print("There are missing values in train dataframe")
else:
    print("Missing Data in the train dataset",df_train.isnull().sum())
if df_test.isnull().any().any()==False:
    print("There are no missing values in test dataframe")
else:
    print("Missing Data i the test dataset",df_test.isnull().sum())

There are missing values in train dataframe
There are no missing values in test dataframe


In [ ]:
df_test.head()

In [13]:
# 1. If for any column(s), the variance is equal to zero, then you need to remove those variable(s).

# 3. Apply label encoder.

for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one 
        # value is useless so we drop it
        x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)
x_train.head()

C:\Users\Priti\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Priti\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,X142,X243,X129,X274,X376,X20,X40,X79,X250,X2,...,X102,X275,X257,X300,X150,X99,X287,X343,X312,X176
0,1,0,0,0,0,0,0,0,0,213,...,0,1,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,215,...,0,1,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,1,110,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,110,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,110,...,0,0,0,0,1,0,0,0,0,0


In [14]:
print('Feature data types:')
x_train[cols].dtypes.value_counts()

Feature data types:


int64    376
dtype: int64

In [15]:
# 4. Perform dimensionality reduction.

n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

In [16]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(
        pca2_results_train, 
        y_train, test_size=0.2, 
        random_state=4242)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(pca2_results_test)

C:\Users\Priti\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [17]:
params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

In [18]:
def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

In [19]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [20]:
xgb_model = xgb.train(params, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

[22:02:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:99.14835	valid-rmse:98.26297	train-r2:-58.35295	valid-r2:-67.63754
Multiple eval metrics have been passed: 'valid-r2' will be used for early stopping.

Will train until valid-r2 hasn't improved in 50 rounds.
[10]	train-rmse:81.27651	valid-rmse:80.36433	train-r2:-38.88428	valid-r2:-44.91014
[20]	train-rmse:66.71610	valid-rmse:65.77334	train-r2:-25.87403	valid-r2:-29.75260
[30]	train-rmse:54.86955	valid-rmse:53.88963	train-r2:-17.17751	valid-r2:-19.64393
[40]	train-rmse:45.24492	valid-rmse:44.21995	train-r2:-11.35979	valid-r2:-12.90012
[50]	train-rmse:37.44736	valid-rmse:36.37245	train-r2:-7.46669	valid-r2:-8.40431
[60]	train-rmse:31.14760	valid-rmse:30.01883	train-r2:-4.85761	valid-r2:-5.40575
[70]	train-rmse:26.08679	valid-rmse:24.90901	train-r2:-3.10878	valid-r2:-3.41057
[80]	train-rmse:22.04667	

In [21]:
# 5. Predict your test_df values using XGBoost.

p_test = xgb_model.predict(d_test)

In [22]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('Submission.csv', index=False)

sub.head()

,ID,y
0,1,82.943069
1,2,97.363327
2,3,83.191460
3,4,76.866570
4,5,112.676414


In [25]:
sub

,ID,y
0,1,82.943069
1,2,97.363327
2,3,83.191460
3,4,76.866570
4,5,112.676414
...,...,...
4204,8410,109.648430
4205,8411,100.056808
4206,8413,99.187675
4207,8414,106.760742
